In [ ]:
import sqlite3
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
connection = sqlite3.connect('/content/drive/MyDrive/european_database.sqlite')
cursor = connection.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('divisions',), ('matchs',), ('results',), ('taula',)]


1. How many seasons are there in the database?


In [ ]:
cursor.execute("SELECT * from matchs")
matchs = cursor.fetchall()
names = [description[0] for description in cursor.description]
matchs_df = pd.DataFrame(matchs,columns=names)
print(matchs_df)

       Div        Date        HomeTeam        AwayTeam  FTHG  FTAG FTR  season
0       B1  2020-08-08     Club Brugge       Charleroi   0.0   1.0   A    2021
1       B1  2020-08-08         Antwerp        Mouscron   1.0   1.0   D    2021
2       B1  2020-08-08        Standard   Cercle Brugge   1.0   0.0   H    2021
3       B1  2020-08-09      St Truiden            Gent   2.0   1.0   H    2021
4       B1  2020-08-09         Waregem            Genk   1.0   2.0   A    2021
...     ..         ...             ...             ...   ...   ...  ..     ...
123399  T1  2006-05-14     Galatasaray     Kayserispor   3.0   0.0   H    2006
123400  T1  2006-05-14   Gaziantepspor     Malatyaspor   1.0   0.0   H    2006
123401  T1  2006-05-14  Genclerbirligi        Rizespor   1.0   3.0   A    2006
123402  T1  2006-05-14      Samsunspor  Diyarbakirspor   1.0   0.0   H    2006
123403  T1  2006-05-14     Trabzonspor        Besiktas   1.0   2.0   A    2006

[123404 rows x 8 columns]


In [ ]:
cursor.execute('SELECT COUNT(DISTINCT season) FROM matchs')
print(cursor.fetchall())

[(16,)]


2. How many different countries are there?

In [ ]:
cursor.execute('SELECT * FROM divisions')
divisions = cursor.fetchall()
names = [description[0] for description in cursor.description]
divisions_df = pd.DataFrame(divisions,columns=names)
print(divisions_df.head())


  division              name      country
0       B1       Division 1A      Belgium
1       D1        Bundesliga  Deutschland
2       D2     2. Bundesliga  Deutschland
3       E0    Premier League      England
4       E1  EFL Championship      England


In [ ]:
cursor.execute('SELECT COUNT(DISTINCT country) FROM divisions')
print(cursor.fetchall())


[(11,)]


3. Which countries start with S?

In [ ]:
cursor.execute('SELECT Distinct country FROM  divisions WHERE country like("S%")')
print(cursor.fetchall())

[('Scotland',), ('Spain',)]


4. How many matches are there?

In [ ]:
cursor.execute("SELECT COUNT(FTR) from matchs")
print(cursor.fetchall())

[(123404,)]


5. How many games has it been tied?

In [ ]:
cursor.execute('SELECT COUNT(FTHG and FTAG)  FROM  matchs WHERE FTHG=FTAG')
print(cursor.fetchall())

[(32794,)]


6. What are the 5 games in which the home team has scored more goals in the 2020 season?

In [ ]:
cursor.execute("SELECT * FROM matchs")
cursor.execute('SELECT season,FTAG,FTHG FROM matchs WHERE season=2020 and FTHG>FTAG')
print(cursor.fetchmany(5))

[(2020, 1.0, 2.0), (2020, 0.0, 6.0), (2020, 0.0, 4.0), (2020, 1.0, 3.0), (2020, 1.0, 3.0)]


7. What is the away team's goal average in the 2020 season?

In [ ]:
cursor.execute('SELECT AVG(FTAG) FROM matchs')
print(cursor.fetchall())

[(1.1444037470422352,)]


8. Make a join to get: country,Date,FTHG,FTAG for the matches played in the season
2020

In [ ]:
cursor.execute('SELECT matchs.Date, matchs.FTHG, matchs.FTAG, matchs.season, divisions.country 
FROM matchs 
LEFT JOIN divisions ON matchs.div=divisions.division WHERE season=2020')
print(cursor.fetchall())


[('2019-07-26', 2.0, 1.0, 2020, 'Belgium'), ('2019-07-27', 0.0, 2.0, 2020, 'Belgium'), ('2019-07-27', 0.0, 1.0, 2020, 'Belgium'), ('2019-07-27', 0.0, 2.0, 2020, 'Belgium'), ('2019-07-27', 1.0, 3.0, 2020, 'Belgium'), ('2019-07-28', 1.0, 2.0, 2020, 'Belgium'), ('2019-07-28', 1.0, 1.0, 2020, 'Belgium'), ('2019-07-28', 1.0, 4.0, 2020, 'Belgium'), ('2019-08-02', 6.0, 0.0, 2020, 'Belgium'), ('2019-08-03', 4.0, 0.0, 2020, 'Belgium'), ('2019-08-03', 1.0, 1.0, 2020, 'Belgium'), ('2019-08-03', 3.0, 1.0, 2020, 'Belgium'), ('2019-08-03', 3.0, 1.0, 2020, 'Belgium'), ('2019-08-04', 6.0, 1.0, 2020, 'Belgium'), ('2019-08-04', 0.0, 0.0, 2020, 'Belgium'), ('2019-08-04', 4.0, 1.0, 2020, 'Belgium'), ('2019-08-09', 0.0, 0.0, 2020, 'Belgium'), ('2019-08-10', 0.0, 2.0, 2020, 'Belgium'), ('2019-08-10', 1.0, 3.0, 2020, 'Belgium'), ('2019-08-10', 1.0, 1.0, 2020, 'Belgium'), ('2019-08-10', 0.0, 2.0, 2020, 'Belgium'), ('2019-08-11', 2.0, 1.0, 2020, 'Belgium'), ('2019-08-11', 2.0, 1.0, 2020, 'Belgium'), ('2019-08-

9. Do a join and groupby to get: country, division, # of games played during the
2020 season.

In [ ]:
cursor.execute('SELECT divisions.country, divisions.division, matchs.FTR 
FROM divisions 
LEFT JOIN matchs ON divisions.division=matchs.Div 
WHERE season=2020 GROUP BY divisions.country')
print(cursor.fetchall())

[('Belgium', 'B1', 'H'), ('Deutschland', 'D2', 'H'), ('England', 'EC', 'H'), ('France', 'F2', 'H'), ('Greece', 'G1', 'H'), ('Italy', 'I2', 'H'), ('Netherlands', 'N1', 'H'), ('Portugal', 'P1', 'H'), ('Scotland', 'SC2', 'H'), ('Spain', 'SP2', 'H'), ('Turkey', 'T1', 'H')]
